In [ ]:
"""EKSİK PREPARATİON TIME OLAN TARİFLERİN ALLRECİPES/GENİUSKİTCHEN/FOOD.COM SİTESİNDEKİ SAYFALARININ URLSİNİ RECİPEDB'DEN ÇEKEN KOD"... """


import requests
import csv
from bs4 import BeautifulSoup
import concurrent.futures


dataset=[]
header_dataset=["Recipe ID","Recipe Title","Continent","Geocultural Region","Country","ISO-3 Code","GDP per capita ($)","Region","Preparation Time","Cooking Time","Ready In","Preparation Time Text","Calories (kCal)","Protein (g)","Carbohydrates (g)","Total fats (g)","Ingredients","RecipeDB URL","Source Type","Source URL"]
header=["Recipe ID","RecipeDB URL","Source URL","Source Type"]

"""liste={  {id: ... , {"Source Type":...., "Source URL":...   }}

}"""

out_dict={}
with open("v2_17.05.2024_0300.csv","r",encoding="utf-8") as file:
    reader=csv.DictReader(file,fieldnames=header)
    next(reader)
    for satir in reader:
        source_type=satir["Source Type"]
        source_url=satir["Source URL"]
        recipeid=satir["Recipe ID"]

        key_dict={"Source Type": source_type,"Source URL":source_url}
        out_dict[recipeid]=key_dict

bilgiler=[]
with open("READY IN DEĞERİ OLMAYANLAR.csv","r",encoding="utf-8") as infile:
    reader=csv.DictReader(infile,fieldnames=header_dataset)
    next(reader)
    for satir in reader:
        if satir["Recipe ID"] in out_dict.keys():
            value=out_dict[satir["Recipe ID"]]
            satir["Source Type"]=value["Source Type"]
            satir["Source URL"]=value["Source URL"]
            bilgiler.append(satir)
        else:
            bilgiler.append(satir)
with open("v2_READY_IN_DEGERİ_OLMAYANLAR.csv","w",newline="",encoding="utf-8") as outfile:
    writer=csv.DictWriter(outfile,fieldnames=header_dataset)
    writer.writeheader()
    for satir in bilgiler:
        writer.writerow(satir)


dataset=[]

header=['Recipe ID', 'Recipe Title', 'Continent', 'Geocultural Region', 'Country', 'ISO-3 Code', 'GDP per capita ($)', 'Region', 'Preparation Time', 'Cooking Time', 'Ready In', 'Preparation Time Text', 'Calories (kCal)', 'Protein (g)', 'Carbohydrates (g)', 'Total fats (g)', 'Ingredients', 'RecipeDB URL', 'Source Type', 'Source URL']
sozluk={}
output_csv="17.05.2024_0300.csv"

with open("READY IN DEĞERİ OLMAYANLAR.csv","r",encoding="utf-8") as infile:
   reader=csv.DictReader(infile,fieldnames=header)
   next(reader)
   for satir in reader:
       if satir["Source URL"] =="NaN":
           dataset.append(satir)

for element in dataset:
    sozluk[element["Recipe ID"]]=element["RecipeDB URL"]

for key in sozluk.keys():
    url=sozluk[key]

def scrape(recipeid):
    print(recipeid)
    url=sozluk[recipeid]

    response=requests.get(url)
    if response.status_code==200:
        soup=BeautifulSoup(response.text,'html.parser')
        collection_li=soup.find('ul',class_='collection')

        if collection_li:
            link_elements=collection_li.find_all('li', class_="collection-item avatar")
            if len(link_elements)>2:
                link_element=link_elements[2]
                a_tag = link_element.find('a')
                if a_tag:
                    source_url=a_tag["href"]
                    link_text = a_tag.get_text(strip=True).replace('\n', '').replace('  ', ' ')


                    if source_url:
                        if link_text:
                            with open(output_csv,"a",newline="",encoding="utf-8") as file:
                                writer=csv.writer(file)
                                writer.writerow([recipeid,url,source_url,link_text])

                        else:
                            with open(output_csv,"a",newline="",encoding="utf-8") as file2:
                                writer2=csv.writer(file2)
                                writer2.writerow([recipeid,url,source_url,"NaN"])

                    else:
                        if link_text:
                            with open(output_csv,"a",newline="",encoding="utf-8") as file3:
                                writer3=csv.writer(file3)
                                writer3.writerow([recipeid,url,"NaN",link_text])

                        else:
                            with open(output_csv,"a",newline="",encoding="utf-8") as file4:
                                writer4=csv.writer(file4)
                                writer4.writerow([recipeid,url,"NaN","NaN"])

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(scrape, sozluk.keys())






In [ ]:
"""ÇEKTİĞİ SAYFA URLLERİNE İSTEK ATARAK READY IN DEĞERLERİNİ ÇEKİP AYRI BİR CSV DOSYASINA YAZAN KOD..."""

import csv
from bs4 import BeautifulSoup
import requests
import concurrent.futures

try:
    sozluk = {}
    header_dataset = ['Recipe ID', 'Recipe Title', 'Continent', 'Geocultural Region', 'Country', 'ISO-3 Code',
                      'GDP per capita ($)', 'Region', 'Preparation Time', 'Cooking Time', 'Ready In',
                      'Preparation Time Text', 'Calories (kCal)', 'Protein (g)', 'Carbohydrates (g)', 'Total fats (g)',
                      'Ingredients', 'RecipeDB URL', 'Source Type', 'Source URL']
    csv_file_path = r"C:\Users\MONSTER\Desktop\PROJ201 Görselleştirme ve ML\PROJ201 Data Cleaning Works\v2_READY_IN_DEGERİ_OLMAYANLAR.csv"

    with open(csv_file_path, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file, fieldnames=header_dataset)
        # İlk satırı oku ve atla (başlık satırı)
        next(reader, None)
        for satir in reader:
            if satir["Source URL"] != "NaN":
                sozluk[satir["Recipe ID"]] = [satir["Source URL"], satir["Source Type"], satir["RecipeDB URL"]]

except FileNotFoundError:
    print(f"Error: The file at path {csv_file_path} was not found.")
    exit(1)
except PermissionError:
    print(f"Error: Permission denied for file at path {csv_file_path}.")
    exit(1)
except StopIteration:
    print(f"Error: The file at path {csv_file_path} is empty or not properly formatted.")
    exit(1)


def scrape(session, recipeid):
    url, source_type, recipedb_url = sozluk[recipeid]

    if source_type == "All Recipes":
        print(f"Skipping All Recipes link for {recipeid}")
        return None

    try:
        response = session.get(url)
        response.raise_for_status()
        print(f"Processing {recipeid} from {url}")
        soup = BeautifulSoup(response.text, "html.parser")

        total_time = None

        if source_type in ["Genius Kitchen", "Genius Kitchen(food.com)", "Food.com"]:
            total_time_div = soup.find('div', class_='mntl-recipe-details__label', string='Total Time:')
            if total_time_div:
                total_time = total_time_div.find_next_sibling('div', class_='mntl-recipe-details__value').get_text(
                    strip=True)
            print(total_time)
        return [recipeid, total_time, url, source_type, recipedb_url] if total_time else None

    except requests.exceptions.RequestException as e:
        print(f"Request error for {recipeid} from {url}: {e}")
        return None
    except Exception as e:
        print(f"General error processing {recipeid} from {url}: {e}")
        return None


def write_results_to_csv(results, output_csv):
    with open(output_csv, "a", newline="", encoding="utf-8") as outfile:
        writer = csv.writer(outfile)
        for result in results:
            writer.writerow(result)
        print(f"{len(results)} records written to {output_csv}")


def main():
    output_csv = "Main Data Ready In data_All Recipes Hariç.csv"
    results = []

    # Başlıkları ekleyerek dosyayı başlat
    with open(output_csv, "w", newline="", encoding="utf-8") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Recipe ID", "Total Time", "Source URL", "Source Type", "RecipeDB URL"])

    with requests.Session() as session:
        with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
            future_to_recipeid = {executor.submit(scrape, session, recipeid): recipeid for recipeid in sozluk.keys()}
            for future in concurrent.futures.as_completed(future_to_recipeid):
                result = future.result()
                if result:
                    results.append(result)
                    # Her 1000 eklemeyi kontrol et ve yazdır
                    if len(results) % 500 == 0:
                        write_results_to_csv(results, output_csv)
                        results.clear()  # Listeyi temizle

    # Kalan sonuçları yaz
    if results:
        write_results_to_csv(results, output_csv)


if __name__ == "__main__":
    main()
